In [1]:
%cd ..
%pwd

/Users/ender_yang/Desktop/Spark/ds-ciss-policy-docs


'/Users/ender_yang/Desktop/Spark/ds-ciss-policy-docs'

In [2]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from lbl2vec import Lbl2TransformerVec
from sentence_transformers import SentenceTransformer
from transformers import AutoModel, AutoTokenizer
from gensim.models.doc2vec import TaggedDocument
from src.categories import *

In [3]:
df = pd.read_parquet('src/data/all_files.parquet')

In [4]:
df.head(5)

,file_name,content
0,海南省交通运输厅关于行政规范性文件清理结果,【法宝引证码】 CLI.12.5647676\n原文链接：https://www.p...
1,重庆市国土房管局关于同意延长北碚区山水香庭,【法宝引证码】 CLI.12.1300415\n原文链接：https://www.p...
2,甘肃省人力资源和社会保障厅、中国银保监会甘,【法宝引证码】 CLI.12.5608533\n原文链接：https://www.p...
3,银川市人民政府关于公布政府规范性文件目录的,【法宝引证码】 CLI.12.469612\n原文链接：https://www.pk...
4,三亚市人民政府办公室关于三亚市海棠湾6号污,【法宝引证码】 CLI.12.5472959\n原文链接：https://www.p...


In [5]:
documents = []
for i in range(len(df)):
    documents.append(df.iloc[i]['content'].split('.html\n')[1].split('\n')[0])

In [6]:
documents = documents[:10]

In [7]:
documents

['海南省交通运输厅关于行政规范性文件清理结果的通报',
 '重庆市国土房管局关于同意延长北碚区山水香庭安置房项目行政确认有效期的通知',
 '甘肃省人力资源和社会保障厅、中国银保监会甘肃监管局关于进一步做好农民工工资保证金以工程保证保险替代工作的通知',
 '银川市人民政府关于公布政府规范性文件目录的通知',
 '三亚市人民政府办公室关于三亚市海棠湾6号污水提升泵站工程项目土地调查有关事项的通知',
 '甘肃省人民政府办公厅关于印发甘肃省产业扶贫专项贷款工程实施意见的通知',
 '海南省人民政府办公厅转发省文化广电出版体育厅等部门关于推动海南省动漫产业发展实施意见的通知',
 '海南省商务厅外贸处关于参加海峡两岸(泉州)农产品采购订货会的通知',
 '银川市建设局关于印发《银川市建筑施工安全质量标准化示范工程评审办法》的通知',
 '财政部新疆专员办关于审核2010棉花年度出疆棉移库费用补贴有关情况的通知(二)']

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
transformer_model = AutoModel.from_pretrained('bert-base-chinese')

tokenizer_law = AutoTokenizer.from_pretrained("thunlp/Lawformer")
model_law = AutoModel.from_pretrained("thunlp/Lawformer")

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint 

In [9]:
def get_bert_embedding(text):
    inputs = tokenizer_law(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # Get embeddings from BERT model
    with torch.no_grad():
        outputs = model_law(**inputs)
    # Use mean pooling to get a single vector for the entire text
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()

In [10]:
labels = get_l1()
label_keywords = []
for label in labels:
    label_keywords.append(get_l2(label))
    
doc_embeddings = [get_bert_embedding(doc) for doc in documents]

label_embeddings = [get_bert_embedding(label) for label in labels]
    

In [11]:
label_keywords

[['农业', '林业', '牧业', '渔业', '土地', '水利', '资源', '能源', '地质矿产', '环境保护'],
 ['改革开放',
  '统计',
  '国有资产',
  '财政',
  '税收',
  '财务',
  '会计',
  '审计',
  '价格',
  '企业',
  '对外经贸',
  '商贸物资',
  '特区',
  '开发区',
  '外商投资企业',
  '个体经济',
  '房地产',
  '测绘',
  '咨询'],
 ['银行', '外汇', '票据', '证券', '租赁', '期货', '保险', '金融综合'],
 ['宪法',
  '国家机关',
  '机关工作',
  '法制工作',
  '律师',
  '公证',
  '民法',
  '合同',
  '知识产权',
  '反不正当竞争',
  '婚姻赡养收养继承',
  '刑法',
  '民事诉讼',
  '经济审判',
  '行政诉讼',
  '刑事诉讼',
  '劳改劳教监所狱政',
  '司法协助',
  '检察业务'],
 ['民政', '劳动工会', '人事规定', '调解与仲裁', '地震', '教育', '体育'],
 ['科技', '语言文字', '文物文史', '文化', '旅游', '广告'],
 ['卫生', '人口与计划生育', '商品检验与动植物检疫'],
 ['民族事务', '华侨事务', '港澳事务', '台湾事务', '宗教事务'],
 ['建设业', '工业管理', '标准化管理和认证认可', '计量', '质量管理和监督', '工商管理'],
 ['交通运输', '仓储', '邮政电讯'],
 ['海关', '社会监控', '公安', '国家安全', '外交外事', '军事']]

In [12]:
label_keywords

# flatten 2d list to 1d
label_keywords = [item for sublist in label_keywords for item in sublist]
len(label_keywords)

92

In [21]:
len(label_keywords)

92

In [22]:
keywords_embeddings_list = []
for keywords in label_keywords:
    keywords_embeddings = get_bert_embedding(keywords)
    keywords_embeddings_list.append(keywords_embeddings)

In [14]:
len(keywords_embeddings_list)

92

In [23]:
for doc_emb in doc_embeddings:
    # calcualte similarity between each document and each label
    similarity = []
    for i, keyword_emb in enumerate(keywords_embeddings_list):
        similarity.append(np.dot(doc_emb, keyword_emb)/(np.linalg.norm(doc_emb)*np.linalg.norm(keyword_emb)))
    print(f'The most similar label is {label_keywords[np.argmax(similarity)]} with similarity score {np.max(similarity)}')

The most similar label is 国家机关 with similarity score 0.6679660677909851
The most similar label is 国家机关 with similarity score 0.6577949523925781
The most similar label is 质量管理和监督 with similarity score 0.664229154586792
The most similar label is 国家机关 with similarity score 0.6580724716186523
The most similar label is 港澳事务 with similarity score 0.673235297203064
The most similar label is 国家机关 with similarity score 0.6501578092575073
The most similar label is 国家机关 with similarity score 0.6497042179107666
The most similar label is 港澳事务 with similarity score 0.6966918706893921
The most similar label is 质量管理和监督 with similarity score 0.6757918000221252
The most similar label is 国家机关 with similarity score 0.7018440961837769


In [13]:
model = Lbl2TransformerVec(transformer_model=transformer_model, keywords_list=label_keywords,
                           documents=documents)
model.fit()

2024-02-26 12:45:00,238 - Lbl2TransformerVec - INFO - Compute keyword embeddings
2024-02-26 12:45:00,238 - Lbl2TransformerVec - INFO - Compute keyword embeddings
2024-02-26 12:45:04,707 - Lbl2TransformerVec - INFO - Compute document embeddings
2024-02-26 12:45:04,707 - Lbl2TransformerVec - INFO - Compute document embeddings


KeyboardInterrupt: 

In [26]:
model.predict_model_docs()

2024-02-26 12:22:52,442 - Lbl2TransformerVec - INFO - Get document embeddings from model
2024-02-26 12:22:52,445 - Lbl2TransformerVec - INFO - Calculate document<->label similarities


,doc_key,most_similar_label,highest_similarity_score,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9,label_10
0,0,label_0,0.787184,0.787184,0.787184,0.787184,0.787184,0.787184,0.787184,0.787184,0.787184,0.787184,0.787184,0.787184
1,1,label_0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,2,label_0,0.862197,0.862197,0.862197,0.862197,0.862197,0.862197,0.862197,0.862197,0.862197,0.862197,0.862197,0.862197


In [25]:
predictions = model.predict(test_sentences)
# Output predictions
for i, pred in enumerate(predictions):
    print(f"Document {i+1} is predicted to be in category: {labels[pred]}")

AttributeError: 'Lbl2TransformerVec' object has no attribute 'predict'